# 1 Introduction

The applications of recommender systems in businesses have become increasingly popular. Recommender systems apply various sources of information including demographics, social, and preferences to provide users with tailored recommended items. Moreover, the favoured technique for building recommender systems is Collaborative filtering. This technique is further divided into three main categories including Memory-Based, Model-Based, and Hybrid-BAsed. Therefore, as large businesses realise the advantages of recommending personalized items to users, the research for techniques, sources of information, and implementation grows. Netflix is among the big organisations interested in the expansion of recommmender system. The application of recommender systems at Netflix is widely known, however improvements are continiously being investigated to provide users with the best movie and series recommendations. Thus, this notebook aims to investigate the application of collaborative filtering techniques for Netflix.      

#### Which type of RecSys based on CF could Netflix user to provide the most accurate recommendations to users?

- What are the different type of RecSys based on CF?
- Which types could be used for Netflix' dataset?
- How do KNN and SVD compare?

# 2 Importing Libraries

Importing the necessary libraries

In [132]:
# General imports
import os
import pandas as pd
import numpy as np

# Used for visualisations in the EDA
import seaborn as sns

# Used in reducing the memory storage of sparse matrices
from scipy.sparse import csr_matrix

# Used for creating a KNN and SVD RecSys model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds

# Used for performance evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# As working with NaN values in matrices overwelmed the output with warnings 
# these warnings will be ignored.
import warnings
warnings.filterwarnings('ignore')

# 3 Data Processing

In [133]:
debugging = True

## 3.1 Netflix Dataset

### 3.1.1 Import Source Data

Code to append movieId to each record in all of the source files if this has not been executed earlier. This will allow all the source files to be loaded into a dataframe with one line of code and without having to add the movieId seperately before concatting the sourcefiles. This additionally resulted in a faster importing time of the source data.

In [134]:
def format_netflix_source():
    # Variable to keep track of which movieId has to be appended
    x = 0
    string = ","+str(x)

    # Loop through each file in the directory
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as f:
            # Check if the first line of the first file already has been formatted
            # If true formatting the source will be skipped
            if(f.readlines()[0] == '1:,1\n'):
                print('Source is already formatted, continuing \n')
                return
            else:
                print('Started formatting source files \n')
                # Append a ',', movieId and \n (newline) to each line resulting in an extra column with movieId when using pandas read_csv 
                file_lines = [''.join([x.strip(), string, '\n']) for x in f.readlines()]
        with open(os.path.join(directory, filename), 'w') as f:
            # Save the formatted file
            f.writelines(file_lines)
            print('Completed formatting source files')
    return

Creating the movie dataframe by concatting all the sourcefiles without their title (skiprows=1). Concluding with naming the columns.

In [135]:
# Define the folder in which all the seperate movie files are located
directory = "/Users/vbraun/Downloads/training_set/"

# Run the function to format the source data if necessary
format_netflix_source()

print('Started concatting all source files to DataFrame \n')
# Performing 1 concat on all sourcefiles to create one dataframe (including itemId)
# As the files do not have a header and differ between each other 'header' = None and the first row will be skipped
netflix_df = pd.concat(pd.read_csv(os.path.join(directory, fname), skiprows=1,header=None) for fname in os.listdir(directory)).rename(columns={0:'userId',1:'rating',2:'date',3:'itemId'})
print('Completed concatting all source files to DataFrame \n')

# Dropping the date column as this is not relevant for this recommender system
netflix_df = netflix_df.drop(columns='date')

display(netflix_df.head(3))

Source is already formatted, continuing 

Started concatting all source files to DataFrame 

Completed concatting all source files to DataFrame 



,userId,rating,itemId
0,1488844,3,1
1,822109,5,1
2,885013,4,1


### 3.1.2 Data Filtering

To allow for faster development a debugging variable is used. If debugging is True the dataset will only consist of the first 100 movies. For the final model, debugging will be set to False.

In [136]:
# In order to achieve faster execution time of the while developing a selection of the total dataset is made when debugging = True
if debugging == True:
    print('Debugging is set to True: limiting the dataset to the first 100 movies')
    filtered_netflix_df = netflix_df[netflix_df['itemId'] <= 100]
else:
    filtered_netflix_df = netflix_df
print('Length of selected dataset: {0} \n'.format(len(filtered_netflix_df)))

Debugging is set to True: limiting the dataset to the first 100 movies
Length of selected dataset: 352771 



In order to filter the dataset based on activity and reduce the sparsity of the data, the data will be grouped and filtered based on movies and users. The datasets will show how many ratings each movie has gotten and how many rating each user has given.

To reduce the sparcity of data in the dataset, we will filter out the users that have rated fewer than 5% of the total amount of movies.

Finally, the movies that have been rated by fewer than 50 people will be filtered out of the dataset.

In [137]:
# Making two dataframes: One grouping items to show how many ratings an item has gotten and one grouping users to show how many ratings an user has given
filtered_movie_count = filtered_netflix_df[['itemId','userId']].groupby('itemId').count().reset_index().rename(columns={'userId':'user_count'})
filtered_user_count = filtered_netflix_df[['itemId','userId']].groupby('userId').count().reset_index().rename(columns={'itemId':'item_count'})

# Defining the minimal percentage of items that a user has to have rated to be included
required_rated_percentage = 0.05
print('Filtering out users that have rated less than {0}% of all movies'.format(int(required_rated_percentage*100)))

# Check if the userId exists when the total amount of items rated by a user (filtered_user_count['item_count']), divided by the total amount of movies (len(filtered_movie_count)' is bigger than the required_rated_percentage
filtered_netflix_df = filtered_netflix_df[filtered_netflix_df['userId'].isin(filtered_user_count[filtered_user_count['item_count']/len(filtered_movie_count) > required_rated_percentage]['userId'])]
print('Length of filtered dataset: {0} \n'.format(len(filtered_netflix_df)))

print('Filtering out movies that have been rated by fewer than {0} users'.format(50))
filtered_netflix_df = filtered_netflix_df[filtered_netflix_df['itemId'].isin(filtered_movie_count[filtered_movie_count['user_count']>50]['itemId'])]
print('Length of filtered dataset:',len(filtered_netflix_df))

Filtering out users that have rated less than 5% of all movies
Length of filtered dataset: 44209 

Filtering out movies that have been rated by fewer than 50 users
Length of filtered dataset: 44209


## 3.2 Jester Dataset

### 3.2.1 Import Source Data

In [138]:
# Creating dataframes for the jester datasets
jester_items = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_items.csv')
jester_ratings = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_ratings.csv')

# Rename jokeId to itemId to have one uniform format for the Netflix and Jester dataset
jester_df = jester_ratings.rename(columns={'jokeId':'itemId'})

# The ratings given in the jester dataset range from -10 to 10. Since calculations including average ratings of users could be influenced by this, the ratings will be increase with 10 to range from 0 - 20. 
jester_df['rating'] = jester_df['rating'] + 10

### 3.2.2 Data filtering

In [139]:
# In order to achieve faster execution time of the while developing a selection of the total dataset is made when debugging = True
if debugging == True:
    print('Debugging is set to True: limiting the dataset to the first 50 jokes')
    filtered_jester_df = jester_df[jester_df['itemId'] <= 50]
else:
    filtered_jester_df = jester_df
print('Length of selected dataset: {0} \n'.format(len(filtered_jester_df)))

Debugging is set to True: limiting the dataset to the first 50 jokes
Length of selected dataset: 715403 



In [140]:
# Making two dataframes: One grouping items to show how many ratings an item has gotten and one grouping users to show how many ratings an user has given
jester_item_count = filtered_jester_df[['itemId','userId']].groupby('itemId').count().reset_index().rename(columns={'userId':'user_count'})
jester_user_count = filtered_jester_df[['itemId','userId']].groupby('userId').count().reset_index().rename(columns={'itemId':'item_count'})

# Defining the minimal percentage of items that a user has to have rated to be included
required_rated_percentage = 0.05
print('Filtering out users that have rated less than {0}% of all jokes'.format(int(required_rated_percentage*100)))

# Check if the userId exists when the total amount of items rated by a user (jester_user_count['item_count']), divided by the total amount of movies (len(jester_item_count)' is bigger than the required_rated_percentage
filtered_jester_df = filtered_jester_df[filtered_jester_df['userId'].isin(jester_user_count[jester_user_count['item_count']/len(jester_item_count) > required_rated_percentage]['userId'])]
print('Length of filtered dataset: {0} \n'.format(len(filtered_jester_df)))

print('Filtering out jokes that have been rated by fewer than {0} users'.format(20))
filtered_jester_df = filtered_jester_df[filtered_jester_df['itemId'].isin(jester_item_count[jester_item_count['user_count']>20]['itemId'])]
print('Length of filtered dataset:',len(filtered_jester_df))

Filtering out users that have rated less than 5% of all jokes
Length of filtered dataset: 711239 

Filtering out jokes that have been rated by fewer than 20 users
Length of filtered dataset: 711239


# 4 Exploratory Data Analysis (EDA)

## 4.1 EDA Netflix Dataset

In [141]:
# print('The filtered dataset has', filtered_df['userId'].nunique(), 'unique users')
# print('The filtered dataset has', filtered_df['itemId'].nunique(), 'unique movies')
# print('The filtered dataset has', filtered_df['rating'].nunique(), 'unique ratings')
# print('The unique ratings are', sorted(filtered_df['rating'].unique()))

In [142]:
# display(filtered_df.head(),filtered_df.tail())

In [143]:
# filtered_df.describe()

In [144]:
# print('Amount of NaN values in the dataset:',filtered_df.loc[lambda x: x.isnull().any(axis=1)].shape[0])

The following graph shows for each movie (as a dot) what its mean rating is in comparison to the total amount of ratings. 

In [145]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('itemId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Movies")
# plt.fig.subplots_adjust(top=0.95)

Notably, movies with low mean ratings have generally been rated a low number of times in relation to movies with a mean rating higher than 3.0.

The next graph shows the same variables as the graph seen above for each user. 

In [146]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('userId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Users")
# plt.fig.subplots_adjust(top=0.95)

The graph for users show that there are several outliers of users that have rated many movies while having a low mean of their ratings. Additionally, the graph shows that mean of the rating mean approaches normality. 

## 4.2 EDA Jester Dataset

In [147]:
# print('The filtered dataset has', filtered_df['userId'].nunique(), 'unique users')
# print('The filtered dataset has', filtered_df['itemId'].nunique(), 'unique movies')
# print('The filtered dataset has', filtered_df['rating'].nunique(), 'unique ratings')
# print('The unique ratings are', sorted(filtered_df['rating'].unique()))

In [148]:
# display(filtered_df.head(),filtered_df.tail())

In [149]:
# filtered_df.describe()

In [150]:
# print('Amount of NaN values in the dataset:',filtered_df.loc[lambda x: x.isnull().any(axis=1)].shape[0])

In [151]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('itemId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Movies")
# plt.fig.subplots_adjust(top=0.95)

In [152]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('userId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Users")
# plt.fig.subplots_adjust(top=0.95)

# 5 Recommender Systems

Both models will be evaluated using the same function that calculates the root mean squared error and the mean absolute error.

In [212]:
def evaluate_predictions(pred, truth):
    # In order to only compare y's that were not 0, a selection is made from y and corresponding the y^ where y != 0 by using .nonzero() 
    pred = pred[truth.nonzero()].flatten()
    truth = truth[truth.nonzero()].flatten()

    # Standard RMSE and MAE calculations
    rmse = np.sqrt(mean_squared_error(pred,truth))
    mae = mean_absolute_error(pred,truth)
    
    return rmse, mae

## 5.1 K Nearest Neighbors (KNN)

### 5.1.1 KNN Model

In [154]:
def train_test(filtered_df):
    # Since the data is sparse the pivot_table is performed in the csr_matrix to reduce the required memory 
    sparse_matrix = csr_matrix(filtered_df.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)
    print(sparse_matrix.check_format)

    # To train and evaluate the KNN model the sparse matrix is split into 70% train, 15% validation and 15% test data
    train_data, test_data = train_test_split(sparse_matrix, test_size=.30)
    test_data, validation_data = train_test_split(test_data, test_size=.50)

    return train_data, validation_data, test_data

In [208]:
def calculate_weighted_knn_prediction(train_data, test_data, k = 5, metric = 'cosine', n_neighbors = 20):
    # Create the NearestNeighbors model with the parameters passed in the function
    knn_model = NearestNeighbors(metric=metric,algorithm='brute',n_neighbors=n_neighbors,n_jobs=-1)

    # Fit the model with the training data
    knn_model_fitted = knn_model.fit(train_data.toarray())

    # Find the closest distance with k amount of users for the passed 'test_data' (either the validation or test dataset depending on the stage)
    distance, indices = knn_model_fitted.kneighbors(test_data.toarray(),k)

    # Create a sorted list including the distances and userId for each user in the test_data that we can iterate through
    sorted_distance_list = sorted(list(zip(indices.squeeze().tolist(), distance.squeeze().tolist())), key=lambda x: x[1])
    
    # Transform the train_data to be able to search by index
    users_df = pd.DataFrame(train_data.toarray())

    # We will now iterate through the list calculating the average ratings given for each item by the k most similar users 
    knn_prediction = []
    for i, (idx, dist) in enumerate(sorted_distance_list):
        inv = 1 / np.array(dist)
        sim_users = np.array(users_df[users_df.index.isin(idx)])
        masked_ar = np.ma.masked_values(sim_users, 0)
        masked_ar_av = np.ma.average(masked_ar,axis=0,weights=inv)
        # Calculate the mean excluding NaN for each item and turn NaN values back to 0's (for items that have not been rated by all similar users) and add them to the prediction list
        knn_prediction.append(masked_ar_av.data)

    return np.array(knn_prediction)


In [209]:
def calculate_unweighted_knn_prediction(train_data, test_data, k = 5, metric = 'cosine', n_neighbors = 20):
    # Create the NearestNeighbors model with the parameters passed in the function
    knn_model = NearestNeighbors(metric=metric,algorithm='brute',n_neighbors=n_neighbors,n_jobs=-1)

    # Fit the model with the training data
    knn_model_fitted = knn_model.fit(train_data.toarray())

    # Find the closest distance with k amount of users for the passed 'test_data' (either the validation or test dataset depending on the stage)
    distance, indices = knn_model_fitted.kneighbors(test_data.toarray(),k)

    # Create a sorted list including the distances and userId for each user in the test_data that we can iterate through
    sorted_distance_list = sorted(list(zip(indices.squeeze().tolist(), distance.squeeze().tolist())), key=lambda x: x[1])
    
    # Transform the train_data to be able to search by index
    users_df = pd.DataFrame(train_data.toarray())

    # We will now iterate through the list calculating the average ratings given for each item by the k most similar users 
    knn_prediction = []
    for i, (idx, dist) in enumerate(sorted_distance_list):
        # When calculating the mean rating we do not want to include 0's as these are items that have not been rated
        # Therefore, all 0's in the matrix will be converted to NaN values so we can use np.nanmean to exclude these values
        
        # Find the ratings of the k most similar users
        sim_users = np.array(users_df[users_df.index.isin(idx)])
        # Replace 0's with NaN
        sim_users[sim_users == 0] = np.nan
        # Calculate the mean excluding NaN for each item and turn NaN values back to 0's (for items that have not been rated by all similar users) and add them to the prediction list
        knn_prediction.append(np.nan_to_num(np.nanmean(sim_users,axis=0)))
    
    return np.array(knn_prediction)


### 5.1.2 KNN Hyper Parameter Tuning

In [215]:
def hyper_parameter_tuning_knn(train_data, validation_data,weighted=True):
    n_neighbors = [5,10,20,50]
    recommendation_amount = [3,5,10]
    metric = ['euclidean','cosine','minkowski']

    hpt_results = []
    for met in metric:
        for k in recommendation_amount:
            for n in n_neighbors:
                if weighted:
                    rmse, mae = evaluate_predictions(calculate_weighted_knn_prediction(train_data = train_data, test_data = validation_data, metric = met, k = k, n_neighbors = n),validation_data.toarray())
                else:
                    rmse, mae = evaluate_predictions(calculate_unweighted_knn_prediction(train_data = train_data, test_data = validation_data, metric = met, k = k, n_neighbors = n),validation_data.toarray())
                hpt_results.append([rmse,met,k,n])
    print(hpt_results)
    best_parameters_knn = sorted(hpt_results, key=lambda x: x[0])[0]
    print(best_parameters_knn)

    return best_parameters_knn

1. Create NearestNeighbors model
1. Fit the model with train data
1. Use kneighbors to find the k amount of neighbors of the jokes in the test data
1. Calculate the prediction by taking the average score of the k most similar jokes
1. Evaluate the model by comparing the actual ratings with the predicted ratings

Algorithm is set at brute (force) because the inputdata is sparse

## 5.2 Singular Value Decomposition (SVD)

### 5.2.1 SVD Model

Pivot the dataset into a matrix with index='userId', columns='itemId', values='rating' in order to later perform user-based collaborative filtering. Moreover, fill_value = 0 in order to remove NaN values and save them as 0. Finally, the matrix is directly stored as a sparse matrix to save memory, instead of first saving the entire matrix into memory. 

Scipy.sparse.linalg.svds was used to perform a partial singular value decomposition of a sparse matrix. This function allows us to specify 'k' which is the number of singular values and singular vectors that have to be computed. 

In [157]:
def calculate_svd_prediction(data, k = 5):
    # Performing the SVD matrix factorisation giving: u (m x r) orthogonal matrix, 
    # s (r x r) diagonal matrix, and vt(ransposed) (r x n) orthogonal matrix.
    u, s, vt = svds(data.toarray(), k = k)

    # A diagonal matrix has to be created for s in order to recreate a matrix from u, s, and vt
    s_diagonal = np.diag(s)

    # Recreate the matrix by performing matrix multiplications of u, s, and vt
    predictions = np.dot(np.dot(u, s_diagonal), vt)
    
    return predictions

In order to evaluate the performance of the recommendations following SVD we only need the $\hat{y}$ of existing $y$. Therefore, all other values will be filtered out of the prediction dataset by using pred[truth.nonzero()]. Afterwards we are able to evaluate the performance of our model by comparing $\hat{y}$ with their corresponding $y$. 

### 5.2.2 SVD Hyper Parameter Tuning

Different k values lead to different predictions

We will perform hyperparameter tuning to find the k with the lowest rmse. For each k we will perform multiple iterations, in which a random sample of the data will be masked and used to calculate the rmse. The rmse of a k value will be the average rmse of all iterations of that k. 

In [158]:
def hyper_parameter_tuning_svd(dataset, iterations = 10):
    results = []
    print('Calculating the average rmse over {0} iterations'.format(iterations))

    # List with k values that will be tested
    k_list = [1,2,3,4,5,6,7,8,9,10,20,30,50,80]

    # As k has to be 0 < k < min(Matrix.shape) in SVD the values in k_list that are higher are filtered out.
    # The shape of the items in the matrix is passed with csr_matrix(dataset.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values).shape[1]
    k_list = list(filter(lambda num: num < csr_matrix(dataset.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values).shape[1], k_list))
    print(k_list)
    
    for k in k_list:
        rmse_list = []
        for i in range(0,iterations):
            dataset_ex_masked, masked_data = train_test_split(dataset, test_size=.05)

            dataset_ex_masked_csr = csr_matrix(dataset_ex_masked.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)
            masked_data_csr = csr_matrix(masked_data.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)

            rmse, mae = evaluate_predictions(calculate_svd_prediction(dataset_ex_masked_csr,k),masked_data_csr.toarray())
            
            rmse_list.append(rmse)
    
        results.append([k,(sum(rmse_list)/len(rmse_list))])
        print('For k = {0}, the average rmse = {1}'.format(k,(sum(rmse_list)/len(rmse_list))))

    best_parameters_svd = sorted(results, key=lambda x: x[1])[0]
    print('The rmse is lowest for k = {0} at = {1}'.format(best_parameters_svd[0],best_parameters_svd[1]))
    
    return best_parameters_svd
    

# 6 Model Evaluation

## 6.1 Performance Netflix Dataset

### 6.1.1 Results KNN model

In [192]:
knn_train_netflix, knn_validation_netflix, knn_test_netflix = train_test(filtered_netflix_df)

<bound method _cs_matrix.check_format of <5629x100 sparse matrix of type '<class 'numpy.float64'>'
	with 44209 stored elements in Compressed Sparse Row format>>


In [194]:
knn_validation_netflix.shape

(845, 100)

In [214]:
best_params_knn = hyper_parameter_tuning_knn(train_data = knn_train_netflix, validation_data = knn_validation_netflix,weighted = False)

2.7214080892007413 euclidean 3 5
2.7214080892007413 euclidean 3 10
2.7214080892007413 euclidean 3 20
2.7214080892007413 euclidean 3 50
2.614684589167889 euclidean 5 5
2.614684589167889 euclidean 5 10
2.614684589167889 euclidean 5 20
2.614684589167889 euclidean 5 50
2.421636527215574 euclidean 10 5
2.421636527215574 euclidean 10 10
2.421636527215574 euclidean 10 20
2.421636527215574 euclidean 10 50
2.6563508045388495 cosine 3 5
2.6563508045388495 cosine 3 10
2.6563508045388495 cosine 3 20
2.6563508045388495 cosine 3 50
2.5332172153460206 cosine 5 5
2.5332172153460206 cosine 5 10
2.5332172153460206 cosine 5 20
2.5332172153460206 cosine 5 50
2.3332618443976623 cosine 10 5
2.3332618443976623 cosine 10 10
2.3332618443976623 cosine 10 20
2.3332618443976623 cosine 10 50
2.7214080892007413 minkowski 3 5
2.7214080892007413 minkowski 3 10
2.7214080892007413 minkowski 3 20
2.7214080892007413 minkowski 3 50
2.614684589167889 minkowski 5 5
2.614684589167889 minkowski 5 10
2.614684589167889 minkowsk

In [213]:
best_params_knn = hyper_parameter_tuning_knn(train_data = knn_train_netflix, validation_data = knn_validation_netflix,weighted = True)

2.7221396166015643 euclidean 3 5
2.7221396166015643 euclidean 3 10
2.7221396166015643 euclidean 3 20
2.7221396166015643 euclidean 3 50
2.6151099082442384 euclidean 5 5
2.6151099082442384 euclidean 5 10
2.6151099082442384 euclidean 5 20
2.6151099082442384 euclidean 5 50
2.4216003784463744 euclidean 10 5
2.4216003784463744 euclidean 10 10
2.4216003784463744 euclidean 10 20
2.4216003784463744 euclidean 10 50
2.657677213008977 cosine 3 5
2.657677213008977 cosine 3 10
2.657677213008977 cosine 3 20
2.657677213008977 cosine 3 50
2.534596629691698 cosine 5 5
2.534596629691698 cosine 5 10
2.534596629691698 cosine 5 20
2.534596629691698 cosine 5 50
2.3337054056706696 cosine 10 5
2.3337054056706696 cosine 10 10
2.3337054056706696 cosine 10 20
2.3337054056706696 cosine 10 50
2.7221396166015643 minkowski 3 5
2.7221396166015643 minkowski 3 10
2.7221396166015643 minkowski 3 20
2.7221396166015643 minkowski 3 50
2.6151099082442384 minkowski 5 5
2.6151099082442384 minkowski 5 10
2.6151099082442384 minko

In [ ]:
evaluate_predictions(knn_test_netflix.toarray(),calculate_knn_prediction(train_data = knn_train_netflix, test_data = knn_test_netflix, metric = best_params_knn[1], k = best_params_knn[2], n_neighbors = best_params_knn[3]))

(2.4400743788271018, 2.1382619824884253)

### 6.1.2 Results SVD model

In [ ]:
best_params_svd = hyper_parameter_tuning_svd(dataset = filtered_netflix_df, iterations = 10)

Calculating the average rmse over 10 iterations
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 50, 80]
For k = 1, the average rmse = 3.2434582331705064
For k = 2, the average rmse = 3.2584467363066794
For k = 3, the average rmse = 3.1973292358072536
For k = 4, the average rmse = 3.296306602841193
For k = 5, the average rmse = 3.2347705776993414
For k = 6, the average rmse = 3.323296926853523
For k = 7, the average rmse = 3.2517444976996472
For k = 8, the average rmse = 3.3207683277565736
For k = 9, the average rmse = 3.2861598712154247
For k = 10, the average rmse = 3.311579997429663
For k = 20, the average rmse = 3.331291270103035
For k = 30, the average rmse = 3.392597882747438
For k = 50, the average rmse = 3.3416114902192744
For k = 80, the average rmse = 3.3837367392051547
The rmse is lowest for k = 3 at = 3.1973292358072536


### 6.1.3 Comparison KNN & SVD for Netflix

#### 6.1.3.1 Performance

#### 6.1.3.2 Recommendations

## 6.2 Performance Jester Dataset

### 6.2.1 Results KNN model

In [ ]:
knn_train_jester, knn_validation_jester, knn_test_jester = train_test(filtered_jester_df)

<bound method _cs_matrix.check_format of <56350x40 sparse matrix of type '<class 'numpy.float64'>'
	with 710368 stored elements in Compressed Sparse Row format>>


In [ ]:
best_params_knn = hyper_parameter_tuning_knn(train_data = knn_train_jester, validation_data = knn_validation_jester)

9.034779956924568 euclidean 3 5
9.034779956924568 euclidean 3 10
9.034779956924568 euclidean 3 20


KeyboardInterrupt: 

In [ ]:
evaluate_predictions(knn_test_jester.toarray(),calculate_knn_prediction(train_data = knn_train_jester, test_data = knn_test_jester, metric = best_params_knn[1], k = best_params_knn[2], n_neighbors = best_params_knn[3]))

### 6.2.2 Results SVD model

In [ ]:
best_params_svd = hyper_parameter_tuning_svd(dataset = filtered_jester_df, iterations = 2)

### 6.2.3 Comparison KNN & SVD for Jester

#### 6.1.3.1 Performance

#### 6.1.3.2 Recommendations

KNN:

In [ ]:
# recommend_for_user = 50

# user_pred_df = pd.DataFrame(predictions)
# user_sel_pred_df = user_pred_df.loc[recommend_for_user].sort_values(ascending=False)

# user_df = pd.DataFrame(final_csr_matrix.toarray())
# selected = pd.DataFrame(user_df.loc[recommend_for_user])
# rated_movies = selected.loc[~(selected==0).all(axis=1)].index.values.tolist()

# recommended_movies = user_sel_pred_df.loc[~user_sel_pred_df.index.isin(rated_movies)]

# print('Rated items are:',rated_movies)
# print(recommended_movies[:3])

SVD:

In [ ]:

# final_csr_matrix = csr_matrix(filtered_df.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)
# predictions = hyper_svd(final_csr_matrix,best_parameters_svd[0])

In [ ]:
# recommend_for_user = 650

# user_pred_df = pd.DataFrame(predictions)
# user_sel_pred_df = user_pred_df.loc[recommend_for_user].sort_values(ascending=False)

# user_df = pd.DataFrame(final_csr_matrix.toarray())
# selected = pd.DataFrame(user_df.loc[recommend_for_user])
# rated_movies = selected.loc[~(selected==0).all(axis=1)].index.values.tolist()

# recommended_movies = user_sel_pred_df.loc[~user_sel_pred_df.index.isin(rated_movies)]

# print('Rated items are:',rated_movies)
# print(recommended_movies[:3])

In order to answer the subquestion: "How do the KNN and SVD models compare?" we will compare compare the rmse of both models for the same dataset.

# 7 Conclusion

#### Which type of RecSys based on CF could Netflix user to provide the most accurate recommendations to users?

- How do KNN and SVD compare?